In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from difflib import SequenceMatcher
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import re

In [2]:
APP_CLIENT_ID = "668e0c3c0e374ae9baeb91ddcd72d537"
APP_CLIENT_SECRET = "b14c0ec7a011407aa78cb688151a69a7"
CRAWL_MATCH_FOLDER = 'crawl/spotify_matching/'
CRAWL_SPOTIFY_FOLDER = 'crawl/spotify_artists/'
CRAWL_TRACKS_FOLDER = 'crawl/spotify_tracks/'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=APP_CLIENT_ID, client_secret=APP_CLIENT_SECRET))

---
## Match songkick artist with spotify artist
---

In [3]:
with open('data/songkick/data_songkick_artists.csv', encoding='utf-8') as artists_file:
    artists_df = pd.read_csv(artists_file, index_col=0)

artists_df

,artist_name
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk)
/artists/694091-kadebostan,!!!Kadebostan!!!
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako"""
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu..."
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ..."
...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead
/artists/1099500-euuroshima,€urOshima
/artists/9573874-belize-it,≈ Belize ≈
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW


In [4]:
# get the similarity between two strings

def get_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [5]:
# save data at regular interval during process

def save_temp_data(data_df, save_path):
    data_df.to_csv(save_path, encoding='utf-8')
    print(save_path)

In [8]:
# match songkick artist with a spotify artist

def search_spotify_artist(artist_name_lower, results_df):
    # query the artist name through spotify search engine
    results = sp.search(q=artist_name_lower, type='artist', limit=50, market='NO')
    artist_query_list = list()
    
    for result in results['artists']['items']:
        # find the similarity between the spotify name and the query name
        result_name = result['name']
        result_name_lower = result_name.lower()
        similarity = round(get_similarity(result_name_lower, artist_name_lower), 2)
        #print(result_name_lower, artist_name_lower, similarity)
        
        artist_query_list.append({
            'spotify_name': result_name,
            'spotify_id': result['id'],
            'spotify_similarity': similarity,
            'spotify_popularity': result['popularity'],
        })
        
    # if the name yields results
    if artist_query_list:
        # get the spotify artist whose name is closer to the query and whose spotify popularity is the highest
        artist_match = sorted(artist_query_list, key=lambda k: (k['spotify_similarity'], k['spotify_popularity']), reverse=True)[0]
        #print(artist_query_list)
        #print(artist_match)
        for column, value in artist_match.items():
            results_df.loc[idx, column] = value
            
    return results_df

In [45]:
artists_df.index.get_loc(artists_df.loc['/artists/642733-nesseria'].name)

20

In [9]:
# get the spotify id of every artist by searching the artist name on spotify

counter = 0
artists_matched_dict = dict()
artists_matched_df = artists_df.copy()
save_path = CRAWL_MATCH_FOLDER + 'temp_data_artists_matched.csv'

for idx, row in tqdm(artists_df.iloc[:].iterrows()):
    artist_name = row['artist_name']
    artist_name_lower = artist_name.lower().rstrip()  #rstrip removes trailing whitespaces
    #print(artist_name, artist_name_lower)
    
    artists_matched_df = search_spotify_artist(artist_name_lower, artists_matched_df)
    
    time.sleep(1)
    counter += 1
    
    # save data at regular interval
    if counter % 100 == 0:
        save_temp_data(artists_matched_df, save_path)
        pass
    
artists_matched_df

100it [01:51,  1.14s/it]

crawl/spotify_matching/temp_data_artists_matched.csv


200it [03:42,  1.13s/it]

crawl/spotify_matching/temp_data_artists_matched.csv


300it [05:36,  1.13s/it]

...

crawl/spotify_matching/temp_data_artists_matched.csv


32200it [9:55:28,  1.16s/it]

crawl/spotify_matching/temp_data_artists_matched.csv


32231it [9:56:02,  1.11s/it]


,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk),!!!,1mmehjf7eHA10uHMisZGJg,0.30,47.0
/artists/694091-kadebostan,!!!Kadebostan!!!,Kadebostan,6hWRK8VkzbYP9p17M5APJR,0.77,11.0
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako""",NaN,NaN,NaN,NaN
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu...",NaN,NaN,NaN,NaN
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,6wBUn8gMP85n8dPu6LoUcF,0.95,41.0
/artists/1099500-euuroshima,€urOshima,Taro Urashima,5RSxs4Rb82wINvjv5KJ7JK,0.64,36.0
/artists/9573874-belize-it,≈ Belize ≈,≈ Belize ≈,3b8bBMsmGoBd8gYn1pngNc,1.00,18.0
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW,NaN,NaN,NaN,NaN


In [10]:
csv_filename = 'data/spotify/data_spotify_artists_matched_v2.csv'
#artists_matched_df.to_csv(csv_filename, encoding='utf-8')
artists_matched_df

,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk),!!!,1mmehjf7eHA10uHMisZGJg,0.30,47.0
/artists/694091-kadebostan,!!!Kadebostan!!!,Kadebostan,6hWRK8VkzbYP9p17M5APJR,0.77,11.0
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako""",NaN,NaN,NaN,NaN
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu...",NaN,NaN,NaN,NaN
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,6wBUn8gMP85n8dPu6LoUcF,0.95,41.0
/artists/1099500-euuroshima,€urOshima,Taro Urashima,5RSxs4Rb82wINvjv5KJ7JK,0.64,36.0
/artists/9573874-belize-it,≈ Belize ≈,≈ Belize ≈,3b8bBMsmGoBd8gYn1pngNc,1.00,18.0
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW,NaN,NaN,NaN,NaN


In [11]:
# save all the artists without a spotify match

no_match_df = artists_matched_df[artists_matched_df['spotify_id'].isna()]
no_match_df.to_csv('data/spotify/data_spotify_artists_no_match_v2.csv', encoding='utf-8')
no_match_df

,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako""",NaN,NaN,NaN,NaN
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu...",NaN,NaN,NaN,NaN
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ...",NaN,NaN,NaN,NaN
/artists/7809989-and-francis-fruit,& Francis Fruit,NaN,NaN,NaN,NaN
/artists/888062-food-for-ya-soul,*FOOD FOR YA SOUL*,NaN,NaN,NaN,NaN
...,...,...,...,...,...
/artists/7382169-eric-fernandez-choeurs-de-cordoue,Éric Fernandez Choeurs De Cordoue,NaN,NaN,NaN,NaN
/artists/7859544-opfelboum-und-palme,Öpfelboum Und Palme,NaN,NaN,NaN,NaN
/artists/4686513-ubers-meer,Übers Meer,NaN,NaN,NaN,NaN
/artists/4043776-uberreel,überreel,NaN,NaN,NaN,NaN


---
## Try to find spotify artist for unmatched artists
---

In [58]:
with open('data/spotify/data_spotify_artists_multiple_matches_manual.csv', encoding='utf-8') as manual_file:
    multiple_matches_df = pd.read_csv(manual_file, index_col=0)
    multiple_matches_df['match_error'] = multiple_matches_df['match_error'].fillna('')
    
multiple_matches_df

,artist_name,spotify_name,match_error,spotify_id,spotify_popularity,spotify_followers,spotify_listeners
artist_id,,,,,,,
/artists/10119580-22deg-halo,22° Halo,22º Halo,,6p355yiyJc67iSNeWLM8WH,11,1067,1500
/artists/9804639-22-halo,22 Halo,22º Halo,,6p355yiyJc67iSNeWLM8WH,11,1067,1500
/artists/7309389-a-crashed-blackbird-rosehip,A Crashed Blackbird Rosehip,A Crashed Blackbird Called Rosehip,,1NKizqgwDCXtUbMHeqA2Tv,6,88,635
/artists/6506249-a-crashed-blackbird-called-rosehip,A Crashed Blackbird Called Rosehip,A Crashed Blackbird Called Rosehip,,1NKizqgwDCXtUbMHeqA2Tv,6,88,635
/artists/5990259-aspell,A-Spell,A.Spell,,4zkmCFtAa1ETWHoZ2sNGvO,0,25,9
...,...,...,...,...,...,...,...
/artists/531180-za,Za!,Zaz,X,1mbgj8ERPs8lWi7t5cYrdy,65,1195585,2100000
/artists/25827-zippo,Zippo,Zippo,,7Int2nVSzSBL8UDty3jGAR,23,1363,1300
/artists/7284179-zippo-italy,Zippo,Zippo,,7Int2nVSzSBL8UDty3jGAR,23,1363,1300


In [63]:
matches_error_df = multiple_matches_df.loc[multiple_matches_df['match_error'] != '']
matches_error_df

,artist_name,spotify_name,match_error,spotify_id,spotify_popularity,spotify_followers,spotify_listeners
artist_id,,,,,,,
/artists/4323041-aeby,Aeby,Abby,X,4rNhP2V4a54UCel3QaVgGp,24,182,6900
/artists/8375378-agcrew,#AGCrew,Acres,X,3y43zXffnLmhyQD1K4QCmD,29,17632,28100
/artists/183136-alex,Alex,Alex G,X,6lcwlkAjBPSKnFBZjjZFJs,60,201082,1200000
/artists/4635043-alexs,Alex-s,Alex G,X,6lcwlkAjBPSKnFBZjjZFJs,60,201082,1200000
/artists/296901-almanach,Almanach,Almanac,X,2EJxcRlcIa5W1u2v42PvTv,49,48024,333000
...,...,...,...,...,...,...,...
/artists/8984079-vanum,VANUM,Van Morrison,X,44NX2ffIYHr6D4n7RaZF7A,73,2214081,9900000
/artists/10100879-wueh,Wueh!,Wehbba,X,2UzAShzs3DO53bSZqvf8Ri,36,34037,95900
/artists/8680104-whitecut,Whitecut,Whiteout,X,35wRsBUrfQziBPzvJTr9bt,42,1416,274900


In [71]:
# get the spotify id of every artist by searching the artist name on spotify

new_matches_df = matches_error_df.copy()

for idx, row in tqdm(matches_error_df.iloc[:].iterrows()):
    artist_name = row['artist_name']
    artist_name_lower = artist_name.lower().rstrip()  #rstrip removes trailing whitespaces
    spotify_id = row['spotify_id']
    spotify_name = row['spotify_name']
    print(artist_name, artist_name_lower)
    print(spotify_name, spotify_id)
    
    new_matches_df = search_spotify_artist(artist_name_lower, new_matches_df)
    new_spotify_id = new_matches_df.loc[idx, 'spotify_id']
    
    if spotify_id != new_spotify_id:
        new_matches_df.loc[idx, 'new_match'] = 'X'
    else:
        new_matches_df.loc[idx, 'new_match'] = ''
    
    time.sleep(1)
    counter += 1
    
new_matches_df

0it [00:00, ?it/s]

Aeby aeby
Abby 4rNhP2V4a54UCel3QaVgGp
{'spotify_name': 'ABBY', 'spotify_id': '1Ppujw75tt7kUv9gEdrFpz', 'spotify_similarity': 0.75, 'spotify_popularity': 28}


1it [00:01,  1.11s/it]

#AGCrew #agcrew
Acres 3y43zXffnLmhyQD1K4QCmD
{'spotify_name': 'Acres', 'spotify_id': '3y43zXffnLmhyQD1K4QCmD', 'spotify_similarity': 0.67, 'spotify_popularity': 31}


2it [00:02,  1.09s/it]

...

Wolve wolve
Wolves 5vF8F6ItVnaHxwBnnYqrmA
{'spotify_name': 'Wolve', 'spotify_id': '0JAKkAIbs8ygPqznTiSqLx', 'spotify_similarity': 1.0, 'spotify_popularity': 0}


137it [02:35,  1.13s/it]

Za! za!
Zaz 1mbgj8ERPs8lWi7t5cYrdy
{'spotify_name': 'Za!', 'spotify_id': '5id3QE9SvJCxWwHVa0zRcD', 'spotify_similarity': 1.0, 'spotify_popularity': 7}


138it [02:36,  1.14s/it]


,artist_name,spotify_name,match_error,spotify_id,spotify_popularity,spotify_followers,spotify_listeners,spotify_similarity,new_match
artist_id,,,,,,,,,
/artists/4323041-aeby,Aeby,ABBY,X,1Ppujw75tt7kUv9gEdrFpz,28,182,6900,0.75,X
/artists/8375378-agcrew,#AGCrew,Acres,X,3y43zXffnLmhyQD1K4QCmD,31,17632,28100,0.67,
/artists/183136-alex,Alex,ALEX,X,65sChHf6etCvSFdXn8NPUO,50,201082,1200000,1.00,X
/artists/4635043-alexs,Alex-s,Alex S,X,1pXMjL8g30cp8t4AAjxxyH,27,201082,1200000,0.83,X
/artists/296901-almanach,Almanach,Almanac,X,2EJxcRlcIa5W1u2v42PvTv,51,48024,333000,0.93,
...,...,...,...,...,...,...,...,...,...
/artists/8984079-vanum,VANUM,Vanum,X,5fRnNOrEt6z1TDudpbzAcC,11,2214081,9900000,1.00,X
/artists/10100879-wueh,Wueh!,WUEH!,X,1cYvNzzQ1cZB8KVdtOdj4E,0,34037,95900,1.00,X
/artists/8680104-whitecut,Whitecut,Whiteout,X,35wRsBUrfQziBPzvJTr9bt,45,1416,274900,0.88,


In [76]:
new_matches_df = new_matches_df.loc[new_matches_df['new_match'] == 'X']
new_matches_df = new_matches_df.drop(columns=['match_error', 'new_match'])
new_matches_df['spotify_followers'] = ''
new_matches_df['spotify_listeners'] = ''
new_matches_df

,artist_name,spotify_name,spotify_id,spotify_popularity,spotify_followers,spotify_listeners,spotify_similarity
artist_id,,,,,,,
/artists/4323041-aeby,Aeby,ABBY,1Ppujw75tt7kUv9gEdrFpz,28,,,0.75
/artists/183136-alex,Alex,ALEX,65sChHf6etCvSFdXn8NPUO,50,,,1.00
/artists/4635043-alexs,Alex-s,Alex S,1pXMjL8g30cp8t4AAjxxyH,27,,,0.83
/artists/286952-alt,Alt,Alt,3cgqGFgSiqcdcAmVP0RT48,16,,,1.00
/artists/292385-ami,Ami,AMI,6ZQhxROkDyYGsijIBDBrhF,44,,,1.00
...,...,...,...,...,...,...,...
/artists/9070664-use-us,Use,USE,19sJ1AkeUQaRji8KCafk87,15,,,1.00
/artists/8984079-vanum,VANUM,Vanum,5fRnNOrEt6z1TDudpbzAcC,11,,,1.00
/artists/10100879-wueh,Wueh!,WUEH!,1cYvNzzQ1cZB8KVdtOdj4E,0,,,1.00


In [79]:
# get spotify data for each artist

new_matches_data_df = new_matches_df.copy()
counter = 0

for idx, row in tqdm(new_matches_data_df.iloc[:].iterrows()):
    spotify_id = row['spotify_id']
    
    # if artist has a spotify id
    if pd.notna(spotify_id):
        artist_info = sp.artist(spotify_id)
        
        url = artist_info['external_urls']['spotify']
        followers = artist_info['followers']['total']
        genres_list = artist_info['genres']
        
        try:
            listeners = get_spotify_listeners(url)
        except IndexError:
            pass
        
        new_matches_data_df.loc[idx, 'spotify_followers'] = followers
        new_matches_data_df.loc[idx, 'spotify_listeners'] = listeners
        
        genre_count = 1
        for genre in genres_list:
            genre_col_name = 'spotify_genre_' + str(genre_count)
            new_matches_data_df.loc[idx, genre_col_name] = genre
            genre_count += 1

        time.sleep(1)
        counter += 1
        
        # save data at regular interval
        if counter % 100 == 0:
            save_path = CRAWL_SPOTIFY_FOLDER + 'temp_data_spotify_artists.csv'
            #spotify_artists_df.to_csv(save_path, encoding='utf-8')
        
new_matches_data_df

87it [02:19,  1.61s/it]


,artist_name,spotify_name,spotify_id,spotify_popularity,spotify_followers,spotify_listeners,spotify_similarity,spotify_genre_1,spotify_genre_2,spotify_genre_3,spotify_genre_4,spotify_genre_5,spotify_genre_6,spotify_genre_7,spotify_genre_8,spotify_genre_9,spotify_genre_10,spotify_genre_11
artist_id,,,,,,,,,,,,,,,,,,
/artists/4323041-aeby,Aeby,ABBY,1Ppujw75tt7kUv9gEdrFpz,28,14194,17500,0.75,german alternative rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/183136-alex,Alex,ALEX,65sChHf6etCvSFdXn8NPUO,50,28736,304400,1.00,cyberpunk,dark clubbing,darksynth,synthwave,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/4635043-alexs,Alex-s,Alex S,1pXMjL8g30cp8t4AAjxxyH,27,154,15200,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/286952-alt,Alt,Alt,3cgqGFgSiqcdcAmVP0RT48,16,66,241,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/292385-ami,Ami,AMI,6ZQhxROkDyYGsijIBDBrhF,44,25966,97500,1.00,romanian pop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/artists/9070664-use-us,Use,USE,19sJ1AkeUQaRji8KCafk87,15,202,360,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/8984079-vanum,VANUM,Vanum,5fRnNOrEt6z1TDudpbzAcC,11,6745,1300,1.00,voidgaze,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/10100879-wueh,Wueh!,WUEH!,1cYvNzzQ1cZB8KVdtOdj4E,0,185,27,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## Compare spotify matches results between v1 and v2
---

In [26]:
with open('data/spotify/data_spotify_artists_matched_v2.csv', encoding='utf-8') as matched_file:
    artists_matched_v2_df = pd.read_csv(matched_file, index_col=0)
    
with open('data/spotify/data_spotify_artists_matched.csv', encoding='utf-8') as matched_file:
    artists_matched_v1_df = pd.read_csv(matched_file, index_col=0)
    
artists_matched_v2_df

,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk),!!!,1mmehjf7eHA10uHMisZGJg,0.30,47.0
/artists/694091-kadebostan,!!!Kadebostan!!!,Kadebostan,6hWRK8VkzbYP9p17M5APJR,0.77,11.0
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako""",NaN,NaN,NaN,NaN
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu...",NaN,NaN,NaN,NaN
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,6wBUn8gMP85n8dPu6LoUcF,0.95,41.0
/artists/1099500-euuroshima,€urOshima,Taro Urashima,5RSxs4Rb82wINvjv5KJ7JK,0.64,36.0
/artists/9573874-belize-it,≈ Belize ≈,≈ Belize ≈,3b8bBMsmGoBd8gYn1pngNc,1.00,18.0
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW,NaN,NaN,NaN,NaN


In [28]:
# compare matches v1 vs v2

id_v1 = list(artists_matched_v1_df['spotify_id'])
id_v2 = list(artists_matched_v2_df['spotify_id'])
idx_list = list()

for x in range(len(id_v1)):
    if id_v1[x] != id_v2[x]:
        if pd.notna(id_v1[x]) or pd.notna(id_v2[x]):
            idx_list.append(x)

print('Number of new matches:', len(idx_list))
artists_matched_v2_df.iloc[idx_list].merge(artists_matched_v1_df[['spotify_name', 'spotify_id']].iloc[idx_list], right_index=True, left_index=True)

Number of new matches: 2481


,artist_name,spotify_name_x,spotify_id_x,spotify_similarity,spotify_popularity,spotify_name_y,spotify_id_y
/artists/7433784-pyramids-jazz,"""The Pyramids""",The Pyramids,7Krj6vYZibCNyVu9iLqlt8,0.92,28.0,The Pyramids,3kCWVwcSSSggzNsjm8a6h2
/artists/44425-smoove,$MOOVE,$moove,3fPGaOzFjcynEbxfinAGi4,1.00,4.0,$MOOVE,38DshhONySNb65KLMDMCMY
/artists/8672184-and-friends,& Friends,& Friends,2eRxdRJPlhgdDk9SC6UJD2,1.00,0.0,Friends,0R2qhaNYSKYhYKeRMYwrpN
/artists/1301647-meer,**@meer**,MEER,4pAX6VBnbccF9uVcb9EGiq,0.62,21.0,Meer,6DmyMZv3lelGiNWVoceuYj
/artists/5563503-,+++,+++,2omFrAln7XB3qMAc79qR6q,1.00,3.0,NaN,NaN
...,...,...,...,...,...,...,...
/artists/9876474-aecht,Äecht,Acht,4rkDXwv8sSHyb82SlsJD8G,0.67,1.0,Aech,2WfK8cOktfb3xgQNoYny9Z
/artists/9220344-orefik,Ørefik,Orfik,6Zo4265dXQWYtoxdz7k9a7,0.73,0.0,Orezi,6voVBz1bN1mWZCLpNZGqzD
/artists/4808018-uber-you,Über You,Überyou,1bWY1OUdZHaLFpOMtpHM7C,0.93,12.0,User Youth,0cxNfEa3UFPqeMaUudfKtj
/artists/4331501-ubereel,Übereel,Übermensch,75rCRvYp14u4zLfTyv8GCu,0.59,39.0,ubermellow.,5CIUhDzFDyEH61IC2nyfF1


## Get spotify data for each artist
---

In [37]:
with open('data/spotify/data_spotify_artists_matched_v2.csv', encoding='utf-8') as matched_file:
    artists_matched_df = pd.read_csv(matched_file, index_col=0)
    
spotify_artists_df = artists_matched_df.copy()
spotify_artists_df.loc[spotify_artists_df['artist_name'] == 'Tom Pea']

,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity
/artists/4924528-tom-pea,Tom Pea,Tom Pear,0aVSHaOR2StPZJPJRXVDqG,0.93,0.0


In [31]:
# get monthly number of listeners of an artist by scrapping Spotify website (data not available through API)

def get_spotify_listeners(artist_url):
    response = requests.get(artist_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    listeners_text = str(soup.find('meta', {'property': 'og:description'}))
    listeners_nbr = re.findall(r'[0-9]+\.*[0-9]*[MK]*', listeners_text)[0]
    
    # remove K and M and multiply by 1,000 or 1,000,000
    if 'K' in listeners_nbr:
        listeners_nbr = listeners_nbr.replace('K', '')
        listeners_nbr = float(listeners_nbr)*1000
    elif 'M' in listeners_nbr:
        listeners_nbr = listeners_nbr.replace('M', '')
        listeners_nbr = float(listeners_nbr)*1000000
    
    return int(listeners_nbr)

In [32]:
# get spotify related artists for an artist

def get_spotify_related_artists(spotify_id):
    related_artist_count = 1
    related_artists_list = list()
    related_artists_dict = dict()
    
    related_artists = sp.artist_related_artists(spotify_id)
    
    for related_artist in related_artists['artists']:
        related_name = related_artist['name']
        related_id = related_artist['id']
        related_popularity = related_artist['popularity']
        related_artists_list.append({
            'spotify_id': related_id,
            'spotify_name': related_name,
            'spotify_popularity': related_popularity,
        })
    
    # sort list of related artists by popularity (desc.)
    related_artists_list = sorted(related_artists_list, key=lambda k: (k['spotify_popularity']), reverse=True)
    
    # limit to 10 related artists (10 most popular)
    related_artists_list = related_artists_list[:10]

    for related_artist in related_artists_list:
        related_id = related_artist['spotify_id']
        related_id_col_name = 'spotify_related_id_' + str(related_artist_count)
        related_artists_dict[related_id_col_name] = related_id

        related_name = related_artist['spotify_name']
        related_name_col_name = 'spotify_related_name_' + str(related_artist_count)
        related_artists_dict[related_name_col_name] = related_name

        related_artist_count += 1
    
    return related_artists_dict

In [ ]:
# get spotify data for each artist (listeners, followers, genres)

counter = 0

for idx, row in tqdm(spotify_artists_df.iloc[:].iterrows()):
    spotify_id = row['spotify_id']
    
    # if artist has a spotify id
    if pd.notna(spotify_id):
        artist_info = sp.artist(spotify_id)
        
        url = artist_info['external_urls']['spotify']
        followers = artist_info['followers']['total']
        genres_list = artist_info['genres']
        
        try:
            listeners = get_spotify_listeners(url)
        except IndexError:
            pass
        
        spotify_artists_df.loc[idx, 'spotify_followers'] = followers
        spotify_artists_df.loc[idx, 'spotify_listeners'] = listeners
        
        genre_count = 1
        for genre in genres_list:
            genre_col_name = 'spotify_genre_' + str(genre_count)
            spotify_artists_df.loc[idx, genre_col_name] = genre
            genre_count += 1

        time.sleep(1)
        counter += 1
        
        # save data at regular interval
        if counter % 100 == 0:
            save_path = CRAWL_SPOTIFY_FOLDER + 'temp_data_spotify_artists.csv'
            spotify_artists_df.to_csv(save_path, encoding='utf-8')
        
spotify_artists_df.to_csv('data/spotify/data_spotify_artists_v2.csv', encoding='utf-8')
spotify_artists_df

22532it [10:03:16,  1.41s/it]

---
## Get audio features for each artist
---

In [8]:
with open('data/spotify/data_spotify_artists_v2.csv', encoding='utf-8') as artists_file:
    spotify_artists_df = pd.read_csv(artists_file, index_col=0)
    
spotify_artists_df

,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity,spotify_followers,spotify_listeners,spotify_genre_1,spotify_genre_2,spotify_genre_3,...,spotify_genre_9,spotify_genre_10,spotify_genre_11,spotify_genre_12,spotify_genre_13,spotify_genre_14,spotify_genre_15,spotify_genre_16,spotify_genre_17,spotify_genre_18
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk),!!!,1mmehjf7eHA10uHMisZGJg,0.30,47.0,135100.0,305800.0,alternative dance,dance rock,dance-punk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/694091-kadebostan,!!!Kadebostan!!!,Kadebostan,6hWRK8VkzbYP9p17M5APJR,0.77,11.0,1348.0,2000.0,swiss techno,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,6wBUn8gMP85n8dPu6LoUcF,0.95,41.0,64802.0,99200.0,alternative rock,double drumming,indie rock,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/1099500-euuroshima,€urOshima,Taro Urashima,5RSxs4Rb82wINvjv5KJ7JK,0.64,36.0,14600.0,79300.0,okinawan pop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/9573874-belize-it,≈ Belize ≈,≈ Belize ≈,3b8bBMsmGoBd8gYn1pngNc,1.00,18.0,6697.0,4900.0,italian indie pop,milan indie,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# get audio features for a list of tracks

def get_audio_features(tracks_id_list):
    audio_features_dict = dict()
    
    audio_features_list = sp.audio_features(tracks=tracks_id_list)
    for audio_features in audio_features_list:
        if audio_features:
            audio_features_dict[audio_features['id']] = audio_features
        
    return audio_features_dict

In [12]:
# get audio features for each artist's top 10 tracks

tracks_dict = dict()
counter = 0

for idx, row in tqdm(spotify_artists_df.iloc[:].iterrows()):
    spotify_id = row['spotify_id']
    artist_id = idx
    spotify_name = row['spotify_name']
    
    # if artist has a spotify id
    if pd.notna(spotify_id):
        top_tracks = sp.artist_top_tracks(spotify_id)
        tracks_id_list = [track['id'] for track in top_tracks['tracks']]
        audio_features_dict = get_audio_features(tracks_id_list)
        
        for track in top_tracks['tracks']:
            track_name = track['name']
            track_id = track['id']
            track_popularity = track['popularity']
            
            tracks_dict[track_id] = {
                'track_name': track_name,
                'spotify_name': spotify_name,
                'spotify_id': spotify_id,
                'artist_id': artist_id,
                'track_popularity': track_popularity,
            }
            try:
                tracks_dict[track_id].update(audio_features_dict[track_id])
            except KeyError:
                pass
            
        time.sleep(1)
        counter += 1
        
        # save data at regular interval
        if counter % 100 == 0:
            spotify_tracks_df = pd.DataFrame.from_dict(tracks_dict, orient='index')
            save_path = CRAWL_TRACKS_FOLDER + 'temp_data_spotify_tracks.csv'
            spotify_tracks_df.to_csv(save_path, encoding='utf-8')

spotify_tracks_df = pd.DataFrame.from_dict(tracks_dict, orient='index')
spotify_tracks_df.to_csv('data/spotify/data_spotify_tracks_v2.csv', encoding='utf-8')
spotify_tracks_df

32231it [9:37:27,  1.07s/it]


,track_name,spotify_name,spotify_id,artist_id,track_popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
2UjEyfnKzaY7qpBEeESJjv,Even When The Water's Cold,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,49,0.709,0.831,2.0,-6.847,0.0,...,0.1590,0.866,104.971,audio_features,2UjEyfnKzaY7qpBEeESJjv,spotify:track:2UjEyfnKzaY7qpBEeESJjv,https://api.spotify.com/v1/tracks/2UjEyfnKzaY7...,https://api.spotify.com/v1/audio-analysis/2UjE...,227067.0,4.0
7y8aVfDkqt6qirGNivvs0M,One Girl / One Boy,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,46,0.702,0.851,10.0,-5.747,0.0,...,0.3220,0.870,117.004,audio_features,7y8aVfDkqt6qirGNivvs0M,spotify:track:7y8aVfDkqt6qirGNivvs0M,https://api.spotify.com/v1/tracks/7y8aVfDkqt6q...,https://api.spotify.com/v1/audio-analysis/7y8a...,243627.0,4.0
2PAn8J9dbmSnR4fLjpgehC,Dancing Is The Best Revenge,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,43,0.857,0.762,1.0,-6.327,1.0,...,0.0643,0.804,120.010,audio_features,2PAn8J9dbmSnR4fLjpgehC,spotify:track:2PAn8J9dbmSnR4fLjpgehC,https://api.spotify.com/v1/tracks/2PAn8J9dbmSn...,https://api.spotify.com/v1/audio-analysis/2PAn...,281558.0,4.0
0BjOgTLP0kLC88XqMoyrv8,Slyd,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,40,0.858,0.717,0.0,-8.734,1.0,...,0.2160,0.582,120.004,audio_features,0BjOgTLP0kLC88XqMoyrv8,spotify:track:0BjOgTLP0kLC88XqMoyrv8,https://api.spotify.com/v1/tracks/0BjOgTLP0kLC...,https://api.spotify.com/v1/audio-analysis/0BjO...,254920.0,4.0
19plwOsYLdF68361d17aAb,Myth Takes,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,36,0.754,0.770,11.0,-7.830,0.0,...,0.2930,0.904,97.109,audio_features,19plwOsYLdF68361d17aAb,spotify:track:19plwOsYLdF68361d17aAb,https://api.spotify.com/v1/tracks/19plwOsYLdF6...,https://api.spotify.com/v1/audio-analysis/19pl...,143987.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2TU6JSgRLKgks1ydHdPQpC,Wheels in Motion,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,18,0.810,0.378,2.0,-9.397,1.0,...,0.0909,0.320,91.957,audio_features,2TU6JSgRLKgks1ydHdPQpC,spotify:track:2TU6JSgRLKgks1ydHdPQpC,https://api.spotify.com/v1/tracks/2TU6JSgRLKgk...,https://api.spotify.com/v1/audio-analysis/2TU6...,200844.0,4.0
66lXofi3mf385buHSMiKZN,Find Myself a Home,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,16,0.740,0.635,11.0,-9.484,0.0,...,0.0818,0.114,120.498,audio_features,66lXofi3mf385buHSMiKZN,spotify:track:66lXofi3mf385buHSMiKZN,https://api.spotify.com/v1/tracks/66lXofi3mf38...,https://api.spotify.com/v1/audio-analysis/66lX...,236453.0,4.0
5Fl64Ztk3D90gTQbdhZqj0,Devil in Mind,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,13,0.703,0.595,1.0,-9.310,0.0,...,0.1090,0.573,75.863,audio_features,5Fl64Ztk3D90gTQbdhZqj0,spotify:track:5Fl64Ztk3D90gTQbdhZqj0,https://api.spotify.com/v1/tracks/5Fl64Ztk3D90...,https://api.spotify.com/v1/audio-analysis/5Fl6...,271440.0,4.0
1JA12esXthNQiikiQDmbrv,Coz I Luv U,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,13,0.746,0.706,2.0,-6.560,0.0,...,0.1230,0.519,134.147,audio_features,1JA12esXthNQiikiQDmbrv,spotify:track:1JA12esXthNQiikiQDmbrv,https://api.spotify.com/v1/tracks/1JA12esXthNQ...,https://api.spotify.com/v1/audio-analysis/1JA1...,184477.0,4.0


In [13]:
spotify_tracks_df = spotify_tracks_df.drop(columns=['id', 'uri', 'track_href', 'analysis_url', 'type'])
spotify_tracks_df.to_csv('data/spotify/data_spotify_tracks_v2.csv', encoding='utf-8')
spotify_tracks_df

,track_name,spotify_name,spotify_id,artist_id,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
2UjEyfnKzaY7qpBEeESJjv,Even When The Water's Cold,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,49,0.709,0.831,2.0,-6.847,0.0,0.0391,0.10800,0.001280,0.1590,0.866,104.971,227067.0,4.0
7y8aVfDkqt6qirGNivvs0M,One Girl / One Boy,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,46,0.702,0.851,10.0,-5.747,0.0,0.0407,0.00255,0.000059,0.3220,0.870,117.004,243627.0,4.0
2PAn8J9dbmSnR4fLjpgehC,Dancing Is The Best Revenge,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,43,0.857,0.762,1.0,-6.327,1.0,0.0535,0.00339,0.310000,0.0643,0.804,120.010,281558.0,4.0
0BjOgTLP0kLC88XqMoyrv8,Slyd,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,40,0.858,0.717,0.0,-8.734,1.0,0.0995,0.06260,0.243000,0.2160,0.582,120.004,254920.0,4.0
19plwOsYLdF68361d17aAb,Myth Takes,!!!,1mmehjf7eHA10uHMisZGJg,/artists/3684861-chk-chk-chk,36,0.754,0.770,11.0,-7.830,0.0,0.0272,0.13300,0.747000,0.2930,0.904,97.109,143987.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2TU6JSgRLKgks1ydHdPQpC,Wheels in Motion,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,18,0.810,0.378,2.0,-9.397,1.0,0.0596,0.52600,0.009470,0.0909,0.320,91.957,200844.0,4.0
66lXofi3mf385buHSMiKZN,Find Myself a Home,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,16,0.740,0.635,11.0,-9.484,0.0,0.0602,0.53800,0.001940,0.0818,0.114,120.498,236453.0,4.0
5Fl64Ztk3D90gTQbdhZqj0,Devil in Mind,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,13,0.703,0.595,1.0,-9.310,0.0,0.1190,0.00850,0.483000,0.1090,0.573,75.863,271440.0,4.0
1JA12esXthNQiikiQDmbrv,Coz I Luv U,Heymoonshaker,1ar2o79WFRJbq4lci6jq55,/artists/5427898-heymoonshaker,13,0.746,0.706,2.0,-6.560,0.0,0.1140,0.51700,0.000005,0.1230,0.519,134.147,184477.0,4.0


---
### Get list of music genres
---

In [14]:
with open('data/spotify/data_spotify_artists_v2.csv', encoding='utf-8') as artists_file:
    spotify_artists_df = pd.read_csv(artists_file, index_col=0)
    
spotify_artists_df

,artist_name,spotify_name,spotify_id,spotify_similarity,spotify_popularity,spotify_followers,spotify_listeners,spotify_genre_1,spotify_genre_2,spotify_genre_3,...,spotify_genre_9,spotify_genre_10,spotify_genre_11,spotify_genre_12,spotify_genre_13,spotify_genre_14,spotify_genre_15,spotify_genre_16,spotify_genre_17,spotify_genre_18
/artists/3684861-chk-chk-chk,!!! (Chk Chk Chk),!!!,1mmehjf7eHA10uHMisZGJg,0.30,47.0,135100.0,305800.0,alternative dance,dance rock,dance-punk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/694091-kadebostan,!!!Kadebostan!!!,Kadebostan,6hWRK8VkzbYP9p17M5APJR,0.77,11.0,1348.0,2000.0,swiss techno,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/5929284-brothers-in-bamako,"""Brothers In Bamako""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/8704378-floyd-division-the-austrian-pink-floyd-tribute-band,"""Floyd Division"" The Austrian Pink Floyd Tribu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/7172094-jazzetcetera-stewy-von-wattenwyl-group-feat-lisette-spinnler,"""Jazz...etcetera"" Stewy von Wattenwyl Group, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/artists/4301-and-you-will-know-us-by-the-trail-of-dead,…And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,6wBUn8gMP85n8dPu6LoUcF,0.95,41.0,64802.0,99200.0,alternative rock,double drumming,indie rock,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/1099500-euuroshima,€urOshima,Taro Urashima,5RSxs4Rb82wINvjv5KJ7JK,0.64,36.0,14600.0,79300.0,okinawan pop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/9573874-belize-it,≈ Belize ≈,≈ Belize ≈,3b8bBMsmGoBd8gYn1pngNc,1.00,18.0,6697.0,4900.0,italian indie pop,milan indie,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/artists/8889739-luo-chai-cao-yuan-wwww-prairie-wwww,落差草原 WWWW / Prairie WWWW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# get the list of spotify genres

columns_list = ['spotify_genre_' + str(x) for x in range(1, 20)]
genres_df = spotify_artists_df[spotify_artists_df.columns.intersection(columns_list)]
genres_dict = genres_df.stack().value_counts().to_dict()

genres_df = pd.DataFrame.from_dict(genres_dict, orient='index')
genres_df = genres_df.rename(columns={0: 'occurrences'}).sort_index()
genres_df.index.name = 'spotify_genre'

genres_df.to_csv('data/spotify/data_spotify_genres_v2.csv', encoding='utf-8')
genres_df

,occurrences
spotify_genre,
21st century classical,6
5th wave emo,6
a cappella,13
aarhus indie,5
aberdeen indie,1
...,...
zillertal,6
zim urban groove,3
zolo,18
